In [1]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_ollama import ChatOllama
from langchain_community.utilities import SQLDatabase
from pathlib import Path

In [2]:
db_path = Path("..") / "dataset" / "db" / "fitbit.sqlite"

# Add view_support=False here
db = SQLDatabase.from_uri(
    f"sqlite:///{db_path.resolve().as_posix()}",
    view_support=False
)

model = ChatOllama(model="ministral-3:3b")
toolkit = SQLDatabaseToolkit(db=db, llm=model)

tools = toolkit.get_tools()

for tool in tools:
    print(f"{tool.name}: {tool.description}\n")

sql_db_query: Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.

sql_db_schema: Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3

sql_db_list_tables: Input is an empty string, output is a comma-separated list of tables in the database.

sql_db_query_checker: Use this tool to double check if your query is correct before executing it. Always use this tool before executing a query with sql_db_query!



In [17]:
print(f"Dialect: {db.dialect}")
# print(f"Available tables: {db.get_usable_table_names()}")
# query = """
# SELECT total_steps FROM daily_activity WHERE user_id = 1503960366 AND event_date = '2016-04-10';
# """

query = """
SELECT distinct event_date
FROM daily_activity
"""

db.run(query)

Dialect: sqlite


"[('2016-03-25 00:00:00',), ('2016-03-26 00:00:00',), ('2016-03-27 00:00:00',), ('2016-03-28 00:00:00',), ('2016-03-29 00:00:00',), ('2016-03-30 00:00:00',), ('2016-03-31 00:00:00',), ('2016-04-01 00:00:00',), ('2016-04-02 00:00:00',), ('2016-04-03 00:00:00',), ('2016-04-04 00:00:00',), ('2016-04-05 00:00:00',), ('2016-04-06 00:00:00',), ('2016-04-07 00:00:00',), ('2016-04-08 00:00:00',), ('2016-04-09 00:00:00',), ('2016-04-10 00:00:00',), ('2016-04-11 00:00:00',), ('2016-04-12 00:00:00',), ('2016-03-12 00:00:00',), ('2016-03-13 00:00:00',), ('2016-03-14 00:00:00',), ('2016-03-15 00:00:00',), ('2016-03-16 00:00:00',), ('2016-03-17 00:00:00',), ('2016-03-18 00:00:00',), ('2016-03-19 00:00:00',), ('2016-03-20 00:00:00',), ('2016-03-21 00:00:00',), ('2016-03-22 00:00:00',), ('2016-03-23 00:00:00',), ('2016-03-24 00:00:00',)]"

In [7]:
system_prompt = """
You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run,
then look at the results of the query and return the answer. Unless the user
specifies a specific number of examples they wish to obtain, always limit your
query to at most {top_k} results.

You can order the results by a relevant column to return the most interesting
examples in the database. Never query for all the columns from a specific table,
only ask for the relevant columns given the question.

You MUST double check your query before executing it. If you get an error while
executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the
database.

To start you should ALWAYS look at the tables in the database to see what you
can query. Do NOT skip this step.

Then you should query the schema of the most relevant tables.
""".format(
    dialect=db.dialect,
    top_k=5,
)

In [8]:
from langchain.agents import create_agent


agent = create_agent(
    model,
    tools,
    system_prompt=system_prompt,
)

In [10]:
question = "what was the last day i ran?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

what was the last day i ran?
================================== Ai Message ==================================

First, let's explore the database structure to identify relevant tables that might contain information about execution days or runs.
Tool Calls:
  sql_db_list_tables (758700e3-23f8-4d95-918e-07185de3129f)
 Call ID: 758700e3-23f8-4d95-918e-07185de3129f
  Args:
    tool_input:
================================= Tool Message =================================
Name: sql_db_list_tables

clean_daily_activity, clean_heartrate, clean_hourly_steps, clean_weight_log
================================== Ai Message ==================================

The tables retrieved are:
- `clean_daily_activity`
- `clean_heartrate`
- `clean_hourly_steps`
- `clean_weight_log`

To determine the last day of activity, the most relevant table would likely be `clean_daily_activity`, as it suggests daily records. Let's check its s